## C S 329E HW 5

# Crossvalidation and hyperparameter selection

## Group number: 11

## Wenxuan Zhou

For this week's homework we are going to explore the cross validation testing methodology and applying that to get error estimates on the two algorithms we've used so far:
  - Linear Regression
  - Decision Tree classification
  

In [ ]:
# Google colab's default version of scikit-learn isn't the latest, so you will 
# need to update the virtual machine and restart the runtime
#!pip install scikit-learn==1.0

In [ ]:
# import the libraries! If you want to add things here for visualization, please do, 
# but only use sklearn when prompted
import pandas as pd
import numpy as np
from sklearn import tree 
from sklearn.datasets import load_iris
from sklearn.datasets import load_diabetes

# Part 1 - Calculate Generalized Error on Linear Regression with k-fold Cross Validation

## Q1.1 Load in the diabetes data set as a pandas dataframe and series.  
Documentation on the data set is [here](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_diabetes.html).  Name your features dataframe (the independent variables) `df_X` and your target value (the dependent variable) series `s_y`

In [ ]:
(df_X, s_y) = load_diabetes(return_X_y=True, as_frame=True)

## Q1.2 Define a function that creates a linear least squares regression model and a function to predict a continuous value given a linear regression model
The first function should take in two parameters, `df_X`, and `s_y` and return the least squares regression model, $\hat{\beta}$ (using the notation from the ESLII book chapter 3 and HW3).  You can not use any libraries outside of pandas and numpy. Note that the length of beta_hat should be the number of columns in `df_X` + 1. 

The second function should take in two parameters, `beta_hat` - the model generated from the `get_linear_regression` function, and `df_X` - that has the attribute values at which we want to predict a continuous value.  We assume that the format and ordering of `df_X` used to create the model and `df_X` used to generate predictions are consistent. 

In [ ]:
def get_linear_regression_model( df_X, s_y ):
    df_X.insert(0, 'intercept', 1)
    (beta_hat, residuals, rank, s) = np.linalg.lstsq(df_X, s_y, rcond=-1)
    return beta_hat

In [ ]:
# code to check beta_hat
np.random.seed(23)
beta_hat = get_linear_regression_model( pd.DataFrame(np.random.random((34,4))), pd.Series(np.random.random(34)*10.0) )
beta_hat

array([ 4.18818425,  1.77890808,  0.74032569, -1.3506416 ,  0.14535984])

In [ ]:
def predict_linear_regression_value( beta_hat, df_X ):
    df_X.insert(0, 'intercept', 1)
    return df_X.dot(beta_hat)

In [ ]:
predicted_vals = predict_linear_regression_value( beta_hat, pd.DataFrame(np.random.random((3,4))))
predicted_vals

0    4.036695
1    4.395026
2    3.908847
dtype: float64

## Q1.3 Define a function that partitions the dataframe and series data into dictionaries
This function should take in three parameters, `df_X`, `s_y`, and `k`, and returns a tuple of two dictionaries.
In both dictionaries the key is the `k` value (an integer from one to `k` inclusive).
The first dictionary will have the dataframe of the training data that contains approximately $\frac{1}{k}$ of the data (variation due to randomness is acceptable).
The second dictionary will have the series of the target data that contains approximately $\frac{1}{k}$ of the data (variation due to randomness is acceptable). Please note the targets _must match_ the same rows as the dataframe at this index, e.g, the length of the kth partition is the same for the dataframe and series.

Call that function with $k=5$ and create the dictionaries `dict_k_df_X` and `dict_k_s_y`. Print out the number of rows in each fold.  Check that the number of data points in each partition totals the number of data points in the entire dataset. 

Here is some example output from checking the length of the folds:
```
Fold 1 length of dataframe is 88 and length of series is 88
Fold 2 length of dataframe is 96 and length of series is 96
Fold 3 length of dataframe is 88 and length of series is 88
Fold 4 length of dataframe is 79 and length of series is 79
Fold 5 length of dataframe is 91 and length of series is 91
The sum of the number of elements in each fold is 442 and there are 442 rows in the original df
```

In [ ]:
def partition_data( df_X, s_y, k ):
    s_y = pd.DataFrame(s_y)
    dict_k_df_X = {}
    dict_k_s_y = {}
    idx = 0
    split_df_X = np.array_split(df_X, k)
    split_s_y = np.array_split(s_y, k)

    for i in range(1, k+1):
      dict_k_df_X[i] = split_df_X[i-1]
      dict_k_s_y[i] = split_s_y[i-1]

    return (dict_k_df_X, dict_k_s_y)

In [ ]:
(dict_k_df_X, dict_k_s_y) = partition_data( df_X, s_y, 5 )

In [ ]:
# Check fold sizes
total = 0

for key, values in dict_k_df_X.items():
    total += len(values)
    print("Fold " + str(key) + " length of dataframe is " + str(len(values)) + " and length of series is " + str(len(dict_k_s_y[key])))
    
print("The sum of the number of elements in each fold is " + str(total) + " and there are " + str(len(df_X)) + " rows in the original df")

Fold 1 length of dataframe is 89 and length of series is 89
Fold 2 length of dataframe is 89 and length of series is 89
Fold 3 length of dataframe is 88 and length of series is 88
Fold 4 length of dataframe is 88 and length of series is 88
Fold 5 length of dataframe is 88 and length of series is 88
The sum of the number of elements in each fold is 442 and there are 442 rows in the original df


## Q1.4 Define a function that calculates a regression metric
This function should accept two series of equal length $n$ numpy arrays, `s_y`, and `s_y_hat`. The metric it should calculate is the mean absolute error, $MAE = \sum\limits_{i=1}^n\frac{|{s\_y_i - {s\_y\_hat}_i}|}{n}$ 

Test your function by using the vectors:
```
x = np.array([1,2,3])
y = np.array([2,2,3])
```


In [ ]:
def get_mae( s_y, s_y_hat):
    mae = 0
    for i in range(len(s_y)):
      mae += abs(s_y[i] - s_y_hat[i]) / len(s_y)
    return mae

In [ ]:
# Test it 
x = np.array([1,2,3])
y = np.array([2,2,3])
get_mae(x,y)

0.3333333333333333

## Q1.5 Calculate the $MAE$ for each fold
For each fold in your dictionaries, calculate the $MAE$.  Use the partition number key as the test set, and all other partitions as the train set. 

Print the min, max, and mean $MAE$ of your 5 folds. 

You must use your helper functions that you wrote above, `get_linear_regression_model`, `predict_linear_regression_value` and `get_mae`. 

In [ ]:
mae = np.array([])
df_X_train = pd.DataFrame()
s_y_train = pd.DataFrame()

for k in dict_k_df_X.keys():
    
    for j in dict_k_df_X.keys():
      if k == j:
        df_X_test = dict_k_df_X[j]
        s_y_test = dict_k_s_y[j]
      else:
        df_X_train = df_X_train.append(dict_k_df_X[j])
        s_y_train = s_y_train.append(dict_k_s_y[j])

    if 'intercept' in df_X_train.columns:
      df_X_train = df_X_train.drop(['intercept'], axis=1)
    if 'intercept' in df_X_test.columns:
      df_X_test = df_X_test.drop(['intercept'], axis=1)
    beta_hat = get_linear_regression_model(df_X_train, s_y_train)
    s_y_hat = predict_linear_regression_value(beta_hat, df_X_test)
    
    mae = np.append( mae, get_mae(s_y_test.values,s_y_hat.values) )

    df_X_train = pd.DataFrame()
    s_y_train = pd.DataFrame()

In [ ]:
print("The min MAE is {:.2f}, the max MAE is {:.2f}, and the mean MAE is {:.2f}".format(mae.min(),mae.max(),mae.mean()))

The min MAE is 42.39, the max MAE is 48.16, and the mean MAE is 44.28


# Part 2 - Find the best hyperparameter to use in a Decision Tree 

## Q2.1 Load the iris data in as a pandas dataframe and a series
Documentation on the data set is [here](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html). Name your features dataframe (the independent variables) `df_X` and your class label (the dependent variable) series `s_y`

In [ ]:
(df_X, s_y) = load_iris(return_X_y=True, as_frame=True)

## Q2.2 Partition `df_X` and `s_y` into $5$ partitions of roughly equal size
Make 2 dictionaries, with the key of each dictionary the fold number.  The value of the dictionary `dict_k_df_X` is the $k^{th}$ partition of the data, and the value of the dictionary `dict_k_s_y` is the corresponding $k^{th}$ target classification.  Print out the number of rows in each fold.  Check that the number of data points in each partition totals the number of data points in the entire dataset. 

Note, you can reuse the functions from Section 1. 

In [ ]:
(dict_k_df_X, dict_k_s_y) = partition_data(df_X, s_y, 5)

## Q2.3 Define a function that calculates accuracy
The function should accept two series and compare each element for equality.  The accuracy is the number of equal elements divided by the total number of elements.

Test your accuracy function by calling it with the `s_y` loaded from the iris data set and an array of the same length containing all $1$ values. 

In [ ]:
def get_acc( s_1, s_2 ):
    n = 0
    for i in range(len(s_1)):
      if s_1[i] == s_2[i]:
        n += 1
    return n / len(s_1)

In [ ]:
get_acc(s_y,np.ones(len(s_y)))

0.3333333333333333

## Q2.4 Using Nested Cross validation, find the best hyperparameter
Use the [Decision Tree Classifier](https://scikit-learn.org/stable/modules/tree.html#classification) class to build a decision tree inside of a 5-fold cross validation loop.  The partitions you already created in 2.2 will be the outer loop.  In the inside loop you should use 4-fold cross validation (so you don't have to partition _again_) to find the best value for `min_impurity_decrease`.  Use the Gini Index as your impurity measure. 
    Calculate the mean accuracy across the 4 folds of your inner loop for all the candidate `min_impurity_decrease` values, and print the value.  Use the array `np.array([0.1,0.25,0.3,0.4])` as the candidates for the best hyperparameter. If there is a tie (two `min_impurity_decrease` values give the same highest accuracy), choose the lowest `min_impurity_decrease` value. 

For each inner loop, select the best `min_impurity_decrease` and train the outer fold training data on using that value. 

For each of the 5 executions of the inner loop, your output should look something like this:
```
Testing 0.10 min impurity decrease
	Average accuracy over 4 folds is 0.95
Testing 0.25 min impurity decrease
	Average accuracy over 4 folds is 0.86
Testing 0.30 min impurity decrease
	Average accuracy over 4 folds is 0.63
Testing 0.40 min impurity decrease
	Average accuracy over 4 folds is 0.27

Best min impurity decrease is 0.1

```

In [ ]:
possible_min_impurity_decrease = np.array([0.1,0.25,0.3,0.4])
df_X_outer = pd.DataFrame()
s_y_outer = pd.DataFrame()
df_X_inner = pd.DataFrame()
s_y_inner = pd.DataFrame()

# Outer loop
outer_acc = np.array([])
for k in dict_k_df_X.keys():
  print("#" + str(k))

  for j in dict_k_df_X.keys():
    if k == j:
      df_X_test = dict_k_df_X[j]
      s_y_test = dict_k_s_y[j]
    else:
        df_X_outer = df_X_outer.append(dict_k_df_X[j])
        s_y_outer = s_y_outer.append(dict_k_s_y[j])

  (dict_innter_df_X, dict_inner_s_y) = partition_data(df_X_outer, s_y_outer, 4)
  this_acc = np.array([])      
    
  for pos_min_impurity in possible_min_impurity_decrease:
    inner_acc = np.array([])
    for m in dict_innter_df_X.keys():
      for n in dict_inner_s_y.keys():
        if m == n:
          df_X_inner_test = dict_innter_df_X[m]
          s_y_inner_test = dict_inner_s_y[m]
        else:
           df_X_inner = df_X_inner.append(dict_innter_df_X[m])
           s_y_inner = s_y_inner.append(dict_inner_s_y[m])

      clf = tree.DecisionTreeClassifier(min_impurity_decrease=pos_min_impurity, criterion='gini')
      clf = clf.fit(df_X_inner, s_y_inner)
      inner_prediction = clf.predict(df_X_inner_test)
      inner_acc = np.append(inner_acc, get_acc(inner_prediction, s_y_inner_test.values))
    print('Testing ' + str(pos_min_impurity) + ' min impurity decrease')
    print('\tAverage accuracy over 4 fold is ' + str(np.round(inner_acc.mean(), 2)))
    this_acc = np.append(this_acc, inner_acc.mean())
    df_X_inner = pd.DataFrame()
    s_y_inner = pd.DataFrame()

  # Inner loop cross validation code here (use 4 folds, where the fold does not include k)
  print('Best min impurity ' + str(possible_min_impurity_decrease[np.argmax(this_acc)]))
  print()

  # Use best min impurity decrease to train model
  clf = tree.DecisionTreeClassifier(min_impurity_decrease=possible_min_impurity_decrease[np.argmax(this_acc)], criterion='gini')
  clf = clf.fit(df_X_outer, s_y_outer)
  outer_prediction = clf.predict(df_X_test)
  acc = get_acc(outer_prediction, s_y_test.values)

  # outer accuracy calculation 
  outer_acc = np.append(outer_acc, acc) # make sure and calculate this_acc in your loop
  print('Outer accuracy ' + str(np.round(acc * 100, 2)) + '%\n')
  df_X_outer = pd.DataFrame()
  s_y_outer = pd.DataFrame()

#1
Testing 0.1 min impurity decrease
	Average accuracy over 4 fold is 0.97
Testing 0.25 min impurity decrease
	Average accuracy over 4 fold is 0.97
Testing 0.3 min impurity decrease
	Average accuracy over 4 fold is 0.81
Testing 0.4 min impurity decrease
	Average accuracy over 4 fold is 0.58
Best min impurity 0.1

Outer accuracy 100.0%

#2
Testing 0.1 min impurity decrease
	Average accuracy over 4 fold is 0.99
Testing 0.25 min impurity decrease
	Average accuracy over 4 fold is 0.83
Testing 0.3 min impurity decrease
	Average accuracy over 4 fold is 0.83
Testing 0.4 min impurity decrease
	Average accuracy over 4 fold is 0.83
Best min impurity 0.1

Outer accuracy 96.67%

#3
Testing 0.1 min impurity decrease
	Average accuracy over 4 fold is 0.99
Testing 0.25 min impurity decrease
	Average accuracy over 4 fold is 0.83
Testing 0.3 min impurity decrease
	Average accuracy over 4 fold is 0.75
Testing 0.4 min impurity decrease
	Average accuracy over 4 fold is 0.42
Best min impurity 0.1

Outer acc

## Q2.5 Show the generalized performance of the classifier 
Show the generalized performance of the classifier by printing the min, max, and mean accuracy of the outer fold test sets.

In [ ]:
print("The min accuracy is {:.2f}, the max accuracy is {:.2f}, and the mean accuracy is {:.2f}".format(outer_acc.min(),outer_acc.max(),outer_acc.mean()))

The min accuracy is 0.83, the max accuracy is 1.00, and the mean accuracy is 0.93
